<a href="https://colab.research.google.com/github/gumindra/image_processing/blob/main/DCT_PNG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image
from scipy.fftpack import dct, idct
import time

def kompresi_gambar(nama_file, kualitas):
  # Mulai mengukur waktu
  start_time = time.time()

  # Muat gambar dan konversi ke warna YCbCr
  img = Image.open(nama_file).convert('YCbCr')

    # Pisahkan gambar menjadi blok 8x8
  lebar, tinggi = img.size
  blok = []
  for y in range(0, tinggi, 8):
    for x in range(0, lebar, 8):
        blok_gambar = img.crop((x, y, x+8, y+8))
        blok.append(blok_gambar)

  # Kompresi setiap blok menggunakan DCT
  blok_kompresi = []
  for blok_gambar in blok:

    # Konversi blok menjadi array numpy
    blok_arr = np.array(blok_gambar, dtype=np.float32)

    # Lakukan DCT
    dct_arr = dct(dct(blok_arr.T, norm='ortho').T, norm='ortho')

    # Potong koefisien berdasarkan parameter kualitas
    skala_kualitas = np.array([1, 1, kualitas], dtype=np.float32)
    dct_arr /= skala_kualitas
    dct_arr = np.round(dct_arr)

    # Sisipkan blok yang telah dikompresi ke dalam list
    blok_kompresi.append(dct_arr.astype(np.int16))

  # Rekonstruksi gambar
  blok_rekonstruksi = []
  for blok_kompresi in blok_kompresi:

    # Skalakan koefisien
    blok_kompresi = blok_kompresi * skala_kualitas

    # Lakukan inverse DCT
    idct_arr = idct(idct(blok_kompresi.T, norm='ortho').T, norm='ortho')

    # Konversi kembali ke PIL Image
    idct_arr[idct_arr < 0] = 0
    idct_arr[idct_arr > 180] = 180
    idct_arr = idct_arr.astype(np.uint8)
    idct_gambar = Image.fromarray(idct_arr, mode='YCbCr')
    blok_rekonstruksi.append(idct_gambar)

  # Gabungkan blok-blok menjadi satu gambar
  gabungan_gambar = Image.new('YCbCr', (lebar, tinggi))
  x, y = 0, 0
  for blok_gambar in blok_rekonstruksi:
    gabungan_gambar.paste(blok_gambar, (x, y))
    x += 8
    if x >= lebar:
        x = 0
        y += 8

  # Konversi gambar kembali ke RGB dan simpan dalam format PNG
  gambar_akhir = gabungan_gambar.convert('RGB')
  gambar_akhir.save('kompresi.png', format='PNG')

  # Selesai mengukur waktu
  end_time = time.time()

  # Durasi proses pengkompresan dalam detik
  duration = end_time - start_time

  # Tampilkan durasi proses
  print("Durasi pengkompresan: %.2f detik" % duration)

  kompresi_gambar('test.png', 500)

In [ ]:
kompresi_gambar('test.png', 500)

Durasi pengkompresan: 46.82 detik
Durasi pengkompresan: 46.43 detik
Durasi pengkompresan: 47.48 detik
Durasi pengkompresan: 46.06 detik
Durasi pengkompresan: 46.50 detik
Durasi pengkompresan: 48.11 detik
Durasi pengkompresan: 49.02 detik
Durasi pengkompresan: 63.47 detik
